<a href="https://colab.research.google.com/github/carolinehagood/ds4002-project3/blob/main/CNN_model_hagood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow
#get_ipython().system('pip install numpy==1.24.3')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 29.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.


In [1]:
import tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import requests
from io import BytesIO
import gzip
from io import BytesIO

In [3]:
import tensorflow as tf

# Check for GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
#Downloading url images
file_urls = ['https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02085936-Maltese_dog%20(3).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02088364-beagle%20(1).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02099601-golden_retriever%20(1).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02106662-German_shepherd%20(2).csv.gz',
             'https://github.com/carolinehagood/ds4002-project3/raw/refs/heads/main/Data/n02110958-pug%20(1).csv.gz']
data_frames = []
for url in file_urls:
    response = requests.get(url)
    compressed_file = BytesIO(response.content)  # Create a BytesIO object
    with gzip.open(compressed_file, 'rt') as f:  # Open it as a text file
        df = pd.read_csv(f)  # Read the CSV file into a DataFrame
        data_frames.append(df)

In [5]:
# Combine all DataFrames into a single DataFrame
data_df = pd.concat(data_frames, ignore_index=True)

# Print the DataFrame to check
print(data_df.head(1))

          0         1         2         3        4         5         6  \
0  0.141176  0.133333  0.082353  0.156863  0.14902  0.098039  0.164706   

          7         8         9  ...     49144     49145     49146     49147  \
0  0.156863  0.105882  0.172549  ...  0.223529  0.180392  0.188235  0.172549   

     49148     49149     49150     49151  breed             breed_name  
0  0.12549  0.164706  0.141176  0.098039      0  n02085936-Maltese_dog  

[1 rows x 49154 columns]


In [6]:
if data_df['breed_name'].value_counts()['n02085936-Maltese_dog'] > 100:
    data_df = data_df.drop(
        data_df[data_df['breed_name'] == 'n02085936-Maltese_dog'].iloc[100:].index
    )

In [7]:
data_df['breed_name'] = data_df['breed_name'].str.split('-').str[1]


In [8]:
print(data_df['breed'].unique())  # Check unique values
print(data_df['breed'].isna().sum())  # Check for NaN values

# Ensure breed is an integer column (convert if needed)
data_df['breed'] = data_df['breed'].astype(int)

[0 1 2 3 4]
0


In [9]:
X = data_df.iloc[:, :-2].values  # All columns except the last two
print(X.shape)

(755, 49152)


In [10]:
y = data_df['breed'].values  # Labels are from the 'breed' column
print(y.shape)

(755,)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [12]:
# Set constants

IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS = 128, 128, 3
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 4  #number of dog breeds

In [43]:
from sklearn.preprocessing import LabelEncoder

# Normalize X
X = X / 255.0  # Scale pixel values to [0, 1]
# Reshape X to (height, width, channels) for 128x128 images
X = X.reshape(-1, 128, 128, 3)  # 128x128 images with 3 color channels (RGB)


# Encode labels (y) into integers and then one-hot encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Exclude the Pug breed using its encoded integer value
exclude_label = 4  # The integer value assigned to Pug
mask = y_encoded != exclude_label  # Create a mask for all data not labeled as Pug

# Filter the data
X_filtered = X[mask]
y_filtered = y_encoded[mask]

# Update the one-hot encoded labels for filtered data
y_onehot_filtered = to_categorical(y_filtered, num_classes=NUM_CLASSES)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_onehot_filtered, test_size=0.2, random_state=42
)

data_augmentation = ImageDataGenerator(
    rotation_range=30,                # Slightly reduced rotation to avoid unrealistic images
    width_shift_range=0.2,            # Smaller shifts to retain more of the dog's body in frame
    height_shift_range=0.2,           # Same as width shift for consistency
    brightness_range=[0.9, 1.1],      # Subtle brightness adjustments to mimic lighting variations
    zoom_range=[0.8, 1.2],            # Allow both zoom-in and zoom-out for better feature generalization
    shear_range=0.15,                 # Slight shear, as higher values might distort features
    horizontal_flip=True,             # Keep this for natural variability
    vertical_flip=False,              # Dogs don't appear upside-down naturally
    fill_mode='reflect',              # Better preservation of edge pixels compared to 'nearest'
    preprocessing_function=None       # Optionally, apply custom preprocessing like CLAHE
)

In [44]:
#build cnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Load the pre-trained VGG16 model, excluding the top layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Fit the model
history = model.fit(
    data_augmentation.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 468ms/step - accuracy: 0.2237 - loss: 1.3860 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.2827 - loss: 1.3844 - val_accuracy: 0.2946 - val_loss: 1.3861
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.2637 - loss: 1.3837 - val_accuracy: 0.2946 - val_loss: 1.3863
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.2978 - loss: 1.3820 - val_accuracy: 0.2946 - val_loss: 1.3863
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.2724 - loss: 1.3815 - val_accuracy: 0.2946 - val_loss: 1.3865
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - accuracy: 0.2799 - loss: 1.3815 - val_accuracy: 0.2946 - val_loss: 1.3866
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.3017 - loss: 1.3781 - val_accuracy: 0.2946 - val_loss: 1.3868
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 0.2865 - loss: 1.3769 - val_accuracy: 0.2946 - val_loss:

In [47]:
#unfreeze some layers of the pre-trained base model (fine-tuning)
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Step 3: Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),  # Use a lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Continue training (fine-tuning)
history_finetune = model.fit(
    data_augmentation.flow(X_train, y_train, batch_size=32),  # Augmented training data
    validation_data=(X_test, y_test),  # Validation data
    epochs=20,  # Continue for additional epochs
    initial_epoch=history.epoch[-1],  # Continue from the last trained epoch
    callbacks=[early_stopping]
)

Epoch 11/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 13s 526ms/step - accuracy: 0.2900 - loss: 1.3852 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 12/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step - accuracy: 0.2797 - loss: 1.3850 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 13/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.2835 - loss: 1.3850 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 14/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 129ms/step - accuracy: 0.2733 - loss: 1.3848 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 15/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.2855 - loss: 1.3850 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 16/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.2962 - loss: 1.3850 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 17/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 0.2681 - loss: 1.3850 - val_accuracy: 0.2946 - val_loss: 1.3860
Epoch 18/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.2995 - loss: 1.3846 - val_accurac

In [48]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3283 - loss: 1.3857 
Test Accuracy: 29.46%
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step


In [29]:
# Classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names='4'))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

Classification Report:


InvalidParameterError: The 'target_names' parameter of classification_report must be an array-like or None. Got '4' instead.